一个 1750 亿参数的 GPT-3 模型需要约 700GB 的参数空间，对应的梯度约为 700GB，而优化器状态还需额外的 1400GB，总计需求高达 2.8TB。使用一块 NVIDIA V100 GPU 来训练拥有 1750 亿参数的 GPT-3 模型，大约需要耗时 288 年。


1. 估训练总计算量（FLOPs）
   业界常用近似：
   **训练 FLOPs ≈ 6 × 参数量 N × 训练 tokens 数 T**

* N = 175B
* T ≈ 300B（GPT-3 论文量级）
  → FLOPs ≈ 6 × 175e9 × 300e9 ≈ **3.15×10²³ FLOPs**

2. 查单卡算力并换算时间
   V100（混合精度 FP16 Tensor Core）峰值算力约 **1.2×10¹⁴ FLOPs/s（≈120 TFLOP/s）**

* 理论最少时间 = 3.15e23 / 1.2e14 ≈ **2.625×10⁹ s ≈ 83 年**
* 若考虑实际利用率：

  * 50% 利用率 → **\~166 年**
  * 30% 利用率 → **\~277 年**
    （如果只按 FP32 \~14 TFLOP/s 算，甚至要 **\~700+ 年**。）

3. 可选：若按 “Chinchilla 最优”训练量（T ≈ 20×N，即 \~3.5 兆亿 tokens），计算量会再放大 \~11.7 倍：
* 理论满载也要 **\~970 年**；30% 利用率则是 **\~3200 年** 量级。




**数据并行**

- 把同一个模型副本复制到多块设备（GPU / TPU），把全局批次（global batch）按设备数切成多份；每卡 local batch $b=B/p$；
- 各设备各自前向 / 反向，得到本地梯度 $g_i$；
- 用 All-Reduce 把梯度求和 / 求平均，再统一做一次优化器更新，这样各模型参数保持一致（常用 ring all-reduce，即 `reduce-scatter + all-gather`）
$$\bar g=\frac{1}{p}\sum_i g_i$$
- 更新：每卡用同一个 $\bar g$ 调 `optimizer.step()`

- 典型变体
    - 同步 vs 异步：工业界主流是同步（稳定、易复现）；异步（参数服务器）可减轻“慢卡”拖累，但会有陈旧梯度与收敛波动；
    - Sharded / ZeRO / FSDP：仍属 DP 家族，但把优化器状态 / 梯度 / 参数分片到多卡以降显存占用；
    - Local-SGD / 延迟同步：多步本地更新再同步，降通信，需调参保证收敛；
    - 梯度累计（gradient accumulation）：多次小 batch 叠加成等效的大 B，在最后一次再 all-reduce；同样降通信；

- 优点
    - 几乎不改模型即可扩展；对能放进单卡显存的模型最友好（否则还需要张量并行或流水线并行）；
    - 吞吐可近线性提升（计算占主导、网络足够时，数据分配合理的情况下）；
    - 工程生态成熟：PyTorch DDP、Horovod、DeepSpeed、FSDP 等；

- 局限
    - 通信开销随参数量线性增长：每步需要同步全部梯度；
    - ring all-reduce 下，每卡通信量大约 $\frac{2(p-1)}{p}\times |{\text{grad bytes}}|$；
    - 大 batch 训练的泛化风险：常需线性 / 余弦学习率缩放、warmup、LARS / LAMB 等优化技巧；
    - 慢卡拖累整体（straggler）：同步模式下由最慢设备决定步速；
    - 模型过大放不进单卡显存时，纯 DP 不适用（需配合张量并行 / 流水并行或用 FSDP / ZeRO 分片）；

- 实践要点
    - 用 DDP 而非单机 DataParallel（DDP 通信更高效）；
    - 数据分片采样器：各 rank 读不同切片，epoch 内不重复；保证 shuffle 一致性与可复现；
    - Mixed Precision + 梯度裁剪/缩放，提高吞吐、稳住数值；
    - 批归一化（BN）：多卡需 SyncBN 或避免跨卡统计不一致；
    - 通信优化：梯度 bucket、overlap（反传中分段 all-reduce）、拓扑感知、固定通信规模（梯度融合）；
    - 大 batch 调参：学习率按 $\text{LR}\propto B$ 及 warmup；验证集频率与早停策略要跟上吞吐的提升；

| 特性 | torch.nn.DataParallel（单机） | torch.nn.parallel.DistributedDataParallel（DDP） |
|------|-------------------------------|--------------------------------------------------|
| 运行进程模型 | 单进程 + 多线程（一个 Python 进程，主线程控制多 GPU） | 多进程（通常一 GPU 一进程） |
| 数据/模型复制 | 主 GPU 保存主模型，每步将参数 从主 GPU 复制到其他 GPU | 每个进程都有一份独立模型副本（显存里自己管理） |
| 梯度同步方式 | 反向传播结束后，先收集到主 GPU 再聚合 | 反向过程中分 bucket 边反传边 All-Reduce 同步梯度 |
| 通信框架 | 依赖主进程显存拷贝（PCIe），没有高效分布式通信 | 直接用 NCCL / Gloo 等后端做 GPU-GPU 高速通信 |
| 可扩展性 | 只能单机，多 GPU（性能受 GIL、主卡瓶颈限制） | 单机/多机都能用，支持大规模集群 |
| 性能 | 易用但慢：主 GPU 负担重，CPU 线程调度开销大，GIL 限制明显 | 高性能：几乎线性扩展，支持通信-计算重叠 |
| 工程现状 | 更多是教学/小规模实验用 | 工业界标准做法 |

In [13]:
import numpy as np

grads = [
    np.array([1.0,  0.5, -0.5,  2.0,  1.5,  0.0]),
    np.array([0.0, -1.0,  0.5,  1.0, -0.5,  1.0]),
    np.array([2.0,  0.0,  1.5, -1.0,  0.5,  0.5]),
    np.array([-1.0, 2.0, -0.5, 0.0,  1.0, -1.5]),
]

np.stack(grads, axis=0).sum(axis=0) / np.stack(grads, axis=0).shape[0]

array([0.5  , 0.375, 0.25 , 0.5  , 0.625, 0.   ])

In [14]:
import numpy as np

def all_reduce_average(grads):
    stacked = np.stack(grads, axis=0)
    summed = stacked.sum(axis=0)
    avg = summed / stacked.shape[0]
    return [avg.copy() for _ in grads]

np.random.seed(42)
grads = [
    np.array([1.0,  0.5, -0.5,  2.0,  1.5,  0.0]),
    np.array([0.0, -1.0,  0.5,  1.0, -0.5,  1.0]),
    np.array([2.0,  0.0,  1.5, -1.0,  0.5,  0.5]),
    np.array([-1.0, 2.0, -0.5, 0.0,  1.0, -1.5]),
]

averaged = all_reduce_average(grads)

print("=== DDP All-Reduce Simulation (4 Ranks) ===")
print("\nLocal gradients:")
for i, g in enumerate(grads):
    print(f"Rank {i}: {g}")

print(f"\nGlobal averaged gradient (identical across all ranks):")
print(f"Result: {averaged[0]}")

def bucket_all_reduce_average(grad, world_size, num_buckets=2):
    length = grad.shape[0]
    assert length % num_buckets == 0
    bucket_size = length // num_buckets
    stacked = np.stack(grads, axis=0)
    results = []
    for b in range(num_buckets):
        start = b * bucket_size
        end = start + bucket_size
        bucket_avg = stacked[:, start:end].sum(axis=0) / world_size
        results.append(bucket_avg)
    return np.concatenate(results, axis=0)

bucket_avg = bucket_all_reduce_average(grads[0], world_size=4, num_buckets=2)
print(f"\nBucketed All-Reduce (2 buckets): {bucket_avg}")
print(f"Verification (should match): {np.allclose(bucket_avg, averaged[0])}")

=== DDP All-Reduce Simulation (4 Ranks) ===

Local gradients:
Rank 0: [ 1.   0.5 -0.5  2.   1.5  0. ]
Rank 1: [ 0.  -1.   0.5  1.  -0.5  1. ]
Rank 2: [ 2.   0.   1.5 -1.   0.5  0.5]
Rank 3: [-1.   2.  -0.5  0.   1.  -1.5]

Global averaged gradient (identical across all ranks):
Result: [0.5   0.375 0.25  0.5   0.625 0.   ]

Bucketed All-Reduce (2 buckets): [0.5   0.375 0.25  0.5   0.625 0.   ]
Verification (should match): True


**all-reduce / reduce-scatter / all-gather 都是什么**

集合通信（collective）术语。
- reduce：把所有 rank 的同一块数据按某种运算（通常是求和 sum 运算）合成一个结果，放到一个 rank 上；
- all-gather：每个 rank 有一段不同的数据（分块），把它们收集并广播给所有人，最后每个 rank 都拼出完整向量；
- reduce-scatter：把每个 rank 对应位置的向量先求和（reduce），然后把不同的分块对应再分配回不同的 rank；
- all-reduce：reduce-scatter → all-gather 两阶段；

- 经典 DDP（全量副本）：对梯度做 all-reduce（每卡都要完整梯度）；
- 分片训练（FSDP / ZeRO）：对梯度做 reduce-scatter（每卡只保留自己那一片的“全局和”），后续按需再 all-gather 参数或优化器状态；

- 与反传重叠（overlap）：把梯度按 bucket 分块；某个 bucket 的反传结束就马上发起 reduce-scatter，同时继续计算更底层的反传，提升流水并行度；
- 配合分片优化：分片训练里，reduce-scatter 后每个 rank 直接拥有自己那份已聚合好的梯度分片，正好喂给分片的优化器；等到前向 / 反向需要某层参数时，再按需 all-gather 该层参数，减少常驻显存和无谓通信；

**ring**

设全量梯度向量切成 4 个等长分块：`[块 0, 块 1, 块 2, 块 3]`

- 阶段 A：reduce-scatter（p−1 轮）：每一轮每个 GPU 把自己手里的“分块部分和”发给右邻居，同时从左邻居收来另一个分块并累加上去；p−1 轮后，每个 rank 留下自己编号那块的全局和（Rank r 留下 `块 r` 即可）；
- 阶段 B：all-gather（p−1 轮）：每个 rank 把自己那块全局和按环传播出去，收集到其他三块，拼回完整向量；
- 通信量：每卡总发送 / 接收字节数 ≈ $\frac{2(p-1)}{p}$ ×（消息大小）；延迟约为 (p-1) 次往返，适合大消息、带宽主导场景；



**$\tfrac{2(p-1)}{p}$**

- 设每个 GPU 上完整梯度向量大小为 S（字节）；Ring All-Reduce 会把向量切成 p 个等长分块，每个块大小 S/p；
- Reduce-Scatter 阶段共有 p-1 轮：每轮每个 GPU 只发送 一个分块，大小 S/p；每个 GPU 在本阶段总发送 / 接收字节数 $(p-1)\cdot S/p$；
- All-Gather 阶段也有 p-1 轮：同样每轮一个分块、大小 S/p；每个 GPU 在本阶段总字节数 $(p-1)\cdot S/p$；

- 表示每块卡最终要发送（也等价接收）约 $\tfrac{2(p-1)}{p}$ 倍的自己消息大小的数据；如 p=4，每卡通信量 = $1.5\,S$；当 p 很大时，这个系数趋近 2（靠近带宽最优）；

- 带上延迟模型 $\approx (p-1)\cdot \alpha \;+\; \dfrac{2(p-1)}{p}\cdot \dfrac{S}{B}$，其中 $\alpha$ 是每轮启动 / 往返延迟，$B$ 是链路带宽；

**micro-batch-size 与 global-batch-size 的关系**
- M = micro-batch-size（每张 GPU、每个前传的样本数）；
- DP = 数据并行的进程 / 卡数（比如说 8 张卡，如果 TP=1、PP=1 即不考虑张量和流水线并行）；
- G = global-batch-size（一次优化器更新所对应的全局样本数）；
- 梯度累积步数，每次优化器更新前需要累积多少个 micro-step，必须整除，否则会报错
$$\text{grad\_accum\_steps} \;=\; \frac{G}{M \times DP}$$

比如 M = 12、DP = 8、G = 192 ⇒ grad_accum_steps = 192 / (12×8) = 2 意味着每张卡先连续做 2 次前向+反传累积梯度，再同步做一次 optimizer.step()。
- 每个 micro-step 处理的 token 数 M × seq_len × DP = 12 × 1024 × 8 = 98304
- 每个优化器步处理的 token 数：G × seq_len = 192 × 1024 = 196608

**哪些激活需要保留？**

训练（反向传播需要保留），常见实现中需要保留的激活（会占显存）如下。

* **输入 embedding / token 表示**：`[B, S, d]`。
* **每层 LayerNorm 的必要统计 / 输入**（通常较小）。
* **Attention 的 Q、K、V**（若一次性计算并保留，为 `3 * [B, S, d]`）。
* **Attention 输出 O（合并 heads 后）**：`[B, S, d]`。
* **MLP（FFN）的中间激活**（hidden，常见 `mlp_dim ≈ 4*d`，用于 GELU 等导数）。
* 以及少量临时值（residual、dropout mask、softmax 中间值等，依实现不同）。

> 注：实现细节（是否用 FlashAttention、分步计算 Q/K/V、是否保存 attention scores）会显著影响峰值内存。

推理（自回归 / 带 KV cache）。

* **每层的 Key (K) 和 Value (V) 历史缓存**：这是长期上下文（past tokens）时最大的内存项。合并 heads 后每层约占 `S_past * d`（K） + `S_past * d`（V）项。
* 一般不需要保存 MLP 中间或全部 Q/K/V 中间（可以按 token 增量计算 Q 并与缓存的 K/V 做 attention）。

便捷估算公式（近似）。

* `bytes_per_elem`：每元素字节数（fp16/bf16 = 2，fp32 = 4）
* `L`：层数（transformer block 数）
* `d`：hidden size（model dim）
* `S`：序列长度（tokens）
* `B`：batch size（训练时通常 >1）
* `S_past`：已缓存的 past tokens（推理时）

对于训练（不做激活重计算）近似激活字节数：

```
Activations_bytes ≈ (8 * L + 1) * B * S * d * bytes_per_elem
```

说明：经验系数 `8` 是对 Q/K/V、attn\_out、mlp\_hidden、residuals 等项的合并近似（实现相关，目的是快速估算）。

推理（KV cache）KV cache 字节数（保存 K 和 V）：

```
KV_cache_bytes = 2 * L * S_past * d * bytes_per_elem * B
```

（`2` 表示 K 和 V 两份；如果按 batch=1 常见）